In [1]:
import warnings
import pandas as pd

from utils.LabelsDict import tickers
from utils.load_data import *
from utils.logger import Logger

warnings.filterwarnings("ignore", category=UserWarning, message="Unable to import Axes3D")

from utils.portfolio import Portfolio

log = Logger(__name__).get_logger()

!pip install -r requirements.txt

from pycbrf.toolbox import ExchangeRates, Banks

In [2]:
tickers_list = [
    'GAZP', 'LKOH', 'ROSN',
    'SBER', 'VTBR', 'MOEX',
    'GMKN', 'NLMK', 'RUAL',
    'MTSS', 'RTKM', 'TTLK',
    'MGNT', 'LNTA', 'FESH',
] 

calc = Portfolio(
    dt_calc='2025-05-31',                                     # дата, до которой выгружаются данные
    dt_start='2019-11-03',                                    # максимальная глубина, которая есть на Финам
    stocks_step=10,                                           # указвыаем, что нужны месячные данные
    tickers_list=[
        'GAZP', 'LKOH', 'ROSN',
        'SBER', 'VTBR', 'MOEX',
        'GMKN', 'NLMK', 'RUAL',
        'MTSS', 'RTKM', 'TTLK',
        'MGNT', 'LNTA', 'FESH',
    ] 
)

In [3]:
calc = (
    calc
    .log_system_info()                                        # вывести в лог конфигурацию расчета
    .load_stock_data(                                         # загрузка рыночных данных цен закрытия компаний
        use_backup_data=True,                                 # загружать данные из backup копии
        create_backup=False                                   # обновить backup копию (в случае есть получены новые данные)
    )
    .load_multipliers()                                       # выделить мультипликаторы из МСФО
    .create_portfolio()                                       # создать портфель с загруженными данными
    .adjust_portfolio_data_types()                            # провести настройку форматов дат и типов данных
    .add_macro_data()                                         # загрузить макропараметры
    .fill_missing_values()                                    # обработать пропуски в данных
    .add_dynamic_features()                                   # проставить динамические признаки по дате, например, квартал
    .add_merton_pd()                                          # выполнить расчет вероятности дефолта по формуле Мертона
    .plot_pd_by_tickers(tickers=tickers_list, verbose=False)  # нарисовать графики для вероятностей дефолта
    .plot_stocks(tickers=tickers_list, verbose=False)         # нарисовать динамику котировок акций
    .plot_debt_capitalization(verbose=False)                  # нарисовать динамику долга и стоимости активов
    .calc_irf(impulses_responses = {                          # расчитать функции импульсного отклика
        'inflation': 'PD', 'interest_rate': 'PD', 
        'rubusd_exchange_rate': 'PD', 'unemployment_rate': 'PD'
    }, verbose=False)
    .plot_correlation_matrix(custom_order = [                 # нарисовать матрицу корреляций
        'GAZP', 'LKOH', 'ROSN',                               # Нефтегазовая отрасль
        'SBER', 'VTBR', 'MOEX',                               # Финансовый сектор
        'GMKN', 'NLMK', 'RUAL',                               # Металлургия
        'MTSS', 'RTKM', 'TTLK',                               # Телекоммуникации
        'MGNT', 'LNTA', 'FESH'                                # Розничная торговля
    ], verbose=False)
    .calc_macro_connections()                                 # расчитать регрессию для макропараметров
    .log_completion()                                         # вывести в лог сообщение об окончании расчета
)

2025-06-14 11:50:03,244:utils.portfolio:INFO: ============================================================
2025-06-14 11:50:03,244:utils.portfolio:INFO: ANALYSIS STARTED | Python 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)] | Matplotlib 3.10.3
2025-06-14 11:50:03,260:utils.portfolio:INFO: ============================================================
2025-06-14 11:50:03,261:utils.portfolio:INFO: Stocks data loaded from backup | Records: 855
2025-06-14 11:50:03,560:utils.portfolio:INFO: Multipliers data loaded | Features: ['ticker', 'year', 'quarter', 'EV/EBITDA', 'P/BV', 'P/E', 'P/FCF', 'P/S', 'Долг, млрд руб', 'Долг/EBITDA', 'Капитализация, млрд руб', 'Чистый долг, млрд руб']
2025-06-14 11:50:03,560:utils.portfolio:INFO: Portfolio created | Companies: 15
2025-06-14 11:50:03,576:utils.portfolio:INFO: Column types adjusted: ['Долг, млрд руб', 'Капитализация, млрд руб', 'Чистый долг, млрд руб', 'high', 'low', 'close', 'EV/EBITDA', 'P/BV', 'P/E', 'P/S', '

<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>

In [4]:
calc.portfolio

,ticker,date,time,open,high,low,close,quarter,year,EV/EBITDA,...,debt,Долг/EBITDA,capitalization,interest_rate,inflation,Year,unemployment_rate,rubusd_exchange_rate,quarterly_volatility,PD
796,FESH,2019-12-31,000000,7.70,9.700,7.500,8.81,4,2019,4.83,...,3.290000e+10,2.65,2.600000e+10,0.0625,0.0300,2019,0.046,61.9057,0.4,0.000582
797,FESH,2020-01-31,000000,8.89,9.070,8.110,8.60,1,2020,NaN,...,3.290000e+10,NaN,2.600000e+10,0.0625,0.0240,2020,0.058,63.0359,0.4,0.000582
798,FESH,2020-02-29,000000,8.62,9.390,6.900,7.07,1,2020,NaN,...,3.290000e+10,NaN,2.600000e+10,0.0600,0.0230,2020,0.058,66.9909,0.4,0.000585
799,FESH,2020-03-31,000000,7.29,7.770,4.610,5.74,1,2020,NaN,...,3.290000e+10,NaN,2.600000e+10,0.0600,0.0250,2020,0.058,77.7325,0.4,0.000585
800,FESH,2020-04-30,000000,5.70,7.180,5.610,6.44,2,2020,4.80,...,3.320000e+10,2.81,2.140000e+10,0.0550,0.0310,2020,0.058,73.6894,0.4,0.000897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,VTBR,2024-06-30,000000,99.65,108.800,93.175,105.70,2,2024,NaN,...,1.101000e+12,NaN,5.664000e+11,0.1600,0.0859,2024,0.025,85.7480,0.4,0.001110
291,VTBR,2024-07-31,000000,105.85,106.975,91.650,97.82,3,2024,NaN,...,1.751000e+12,NaN,4.841000e+11,0.1800,0.0913,2024,0.025,86.3300,0.4,0.002410
292,VTBR,2024-08-31,000000,97.80,102.000,91.000,91.48,3,2024,NaN,...,1.751000e+12,NaN,4.841000e+11,0.1800,0.0905,2024,0.025,91.1868,0.4,0.002410
293,VTBR,2024-09-30,000000,90.95,94.600,83.810,89.03,3,2024,NaN,...,1.751000e+12,NaN,4.841000e+11,0.1900,0.0863,2024,0.025,92.7126,0.4,0.002386


In [4]:
import sys
print(sys.executable)

C:\Users\Maxim\AppData\Local\Programs\Python\Python311\python.exe


In [6]:
from pycbrf.toolbox import ExchangeRates, Banks

In [12]:
date_range = pd.date_range(
    start=pd.to_datetime(calc.dt_start, format='%d.%m.%Y'),
    end=pd.to_datetime(calc.dt_calc, format='%d.%m.%Y'),
    freq='D'  # 'D' = день, 'M' = месяц, 'Y' = год
)

In [43]:
import os

In [72]:
def get_rubusd_exchange_rate(
    update_backup: bool=False,
    use_backup: bool=False
) -> pd.DataFrame:

    rubusd_df_path = f'data/macro/rubusd.csv'
    
    if use_backup:
        rates = pd.read_csv(rubusd_df_path)
        print(f'Exchange rates for usd/rub will be use from backup. Last actual date: {rates.date.max()}')
        return rates

    if os.path.exists(rubusd_df_path):
        
        rates = pd.read_csv(rubusd_df_path)
        start_date = rates.date.max()
    else:
        start_date = calc.dt_start

        date_range = pd.date_range(
            start=pd.to_datetime(start_date, format='%d.%m.%Y'),
            end=pd.to_datetime(calc.dt_calc, format='%d.%m.%Y'),
            freq='D' 
        )

        rates_additional = []
        for date in tqdm(date_range[-10:]):
            rates_additional.append(
                (date.strftime('%Y.%m.%d'), float(ExchangeRates(date)['USD'].value))
            )
        
        rates = (
            pd.DataFrame(rates_additional, columns=['date', 'rubusd_exchange_rate'])
            if not os.path.exists(rubusd_df_path)
            else pd.concat(rates, pd.DataFrame(rates_additional, columns=rates.columns))
        )

    if update_backup:
        rates.to_csv(rubusd_df_path, index=False)
        print(f'Backup file for usd/rub exchange rates was updated. New actual date: {rates.date.max()}')

    return rates

In [79]:
get_rubusd_exchange_rate(use_backup=True,update_backup=False)

Exchange rates for usd/rub will be use from backup. Last actual date: 2025.05.31


,date,rubusd_exchange_rate
0,2025.05.22,79.7530
1,2025.05.23,79.7357
2,2025.05.24,79.7108
3,2025.05.25,79.7108
4,2025.05.26,79.7108
5,2025.05.27,79.6588
6,2025.05.28,79.6176
7,2025.05.29,79.6037
8,2025.05.30,78.4970
9,2025.05.31,78.6171


In [38]:
data_path = 

,date,rubusd_exchange_rate
0,2025.05.22,79.7530
1,2025.05.23,79.7357
2,2025.05.24,79.7108
3,2025.05.25,79.7108
4,2025.05.26,79.7108
5,2025.05.27,79.6588
6,2025.05.28,79.6176
7,2025.05.29,79.6037
8,2025.05.30,78.4970
9,2025.05.31,78.6171


In [19]:
from tqdm import tqdm

In [10]:
rates['USD'].value

Decimal('65.5287')

In [6]:
!cp logs/graphs/GAZP_pd.png text/img/GAZP_pd.png
!cp logs/graphs/irf_inflation_PD.png text/img/irf_inflation_PD.png
!cp logs/graphs/irf_interest_rate_PD.png text/img/irf_interest_rate_PD.png
!cp logs/graphs/corr_matrix.png text/img/corr_matrix.png